In [1]:
! pip install mlflow  kfp-kubernetes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 366.8 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 4.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.8 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 10.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
from kfp import dsl, compiler
from kfp.dsl import InputPath, OutputPath, Output
import os

# First two components remain the same
@dsl.component(base_image="python:3.9", packages_to_install=["kagglehub", "pandas", "scikit-learn"])
def download_and_split_data(output_train: OutputPath(), output_val: OutputPath(), output_test: OutputPath()):
    import kagglehub
    import pandas as pd
    from sklearn.model_selection import train_test_split
   
    path = kagglehub.dataset_download("nelgiriyewithana/credit-card-fraud-detection-dataset-2023")
    df = pd.read_csv(f"{path}/creditcard_2023.csv")
   
    train_df, temp_df = train_test_split(df, train_size=0.6, random_state=42)
    val_df, test_df = train_test_split(temp_df, train_size=0.5, random_state=42)
   
    train_df.to_csv(output_train, index=False)
    val_df.to_csv(output_val, index=False)
    test_df.to_csv(output_test, index=False)

@dsl.component(
    base_image="python:3.9",
    packages_to_install=["tensorflow", "scikit-learn", "pandas", "onnx", "onnxruntime", "tf2onnx"]
)
def train_model(train_data: InputPath(), val_data: InputPath(), model_output: OutputPath()):
    import pandas as pd
    import tensorflow as tf
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    import tf2onnx
    import onnx
   
    df_train = pd.read_csv(train_data)
    df_val = pd.read_csv(val_data)
   
    X_train = df_train.drop(['Class', 'id'], axis=1)
    y_train = df_train['Class']
    X_val = df_val.drop(['Class', 'id'], axis=1)
    y_val = df_val['Class']
   
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
   
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(29,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
   
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
   
    class_weights = {0: 1, 1: len(y_train[y_train==0])/len(y_train[y_train==1])}
   
    model.fit(X_train, y_train, epochs=3, validation_data=(X_val, y_val), class_weight=class_weights)

    # # Before conversion, set the output names
    # model.outputs[0]._name = 'output_name'  # Give your output a unique name
   
    # model_proto, _ = tf2onnx.convert.from_keras(model)
    # Convert the Keras model to ONNX
    import tensorflow as tf
    import tf2onnx
    import onnx
    import os
   
    # Wrap the model in a tf.function
    @tf.function(input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name='dense_input')])
    def model_fn(x):
        return model(x)
   
    # Convert the Keras model to ONNX
    model_proto, external_tensor_storage = tf2onnx.convert.from_function(
        model_fn,
        input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name='dense_input')],
        opset=13  # You can specify the ONNX opset version here
    )

    onnx.save(model_proto, model_output)


@dsl.component(
    base_image="python:3.9",
    packages_to_install=["boto3"]
)
def upload_to_minio(
    model_path: InputPath(),
    minio_endpoint: str,
    minio_access_key: str,
    minio_secret_key: str
) -> str:  # Change to return string instead of using Output parameter
    import boto3
    import os
    from botocore.client import Config
   
    # Clean the endpoint URL
    minio_endpoint = minio_endpoint.strip()
   
    # Create S3 client with explicit credentials
    s3_client = boto3.client(
        's3',
        endpoint_url=minio_endpoint,
        aws_access_key_id=minio_access_key,
        aws_secret_access_key=minio_secret_key,
        config=Config(
            signature_version='s3v4',
            s3={'addressing_style': 'path'}
        )
    )
   
    bucket_name = 'mlflow'  # Or your bucket name
    object_name = 'models/fraud/1/model.onnx'
   
    print(f"Uploading {model_path} to {bucket_name}/{object_name}")
   
    # Upload the file
    s3_client.upload_file(
        Filename=model_path,
        Bucket=bucket_name,
        Key=object_name
    )
   
    # Return the MinIO URL
    return f"{minio_endpoint}/{bucket_name}/{object_name}"

@dsl.pipeline(name='fraud-detection-pipeline')
def fraud_detection_pipeline():
    data_op = download_and_split_data()
   
    train_op = train_model(
        train_data=data_op.outputs['output_train'],
        val_data=data_op.outputs['output_val'],
    )
   
    # Get environment variables
    minio_endpoint = 'http://minio.kubeflow.svc.cluster.local:9000'
    minio_access_key = 'admin1234'
    minio_secret_key = 'admin1234'
   
    # Pass credentials explicitly to the component
    minio_url = upload_to_minio(
        model_path=train_op.outputs['model_output'],
        minio_endpoint=minio_endpoint,
        minio_access_key=minio_access_key,
        minio_secret_key=minio_secret_key
    )

if __name__ == '__main__':
    compiler.Compiler().compile(
        pipeline_func=fraud_detection_pipeline,
        package_path='fraud_detection_pipeline.yaml'
    )

In [3]:
import kfp
client = kfp.Client()
client.create_run_from_pipeline_package(
    'fraud_detection_pipeline.yaml',
    arguments={},
    experiment_name='Fraud Detection'
)

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


RunPipelineResult(run_id=e0653d17-fa2d-4348-8ec1-4308c477d69d)